In [21]:
import pandas as pd
import numpy as np

In [194]:
df = pd.read_csv('data/data_table/data_sheet1.csv')
df = df.rename(columns={'Unnamed: 0': 'ranking'})
df = df[df.columns[:25]]

In [195]:
df.columns


Index(['ranking', 'each_original_data', 'ion1', 'loss1', 'loss_sign1',
       'charge1', 'mass1', 'ion2', 'loss2', 'loss_sign2', 'charge2', 'mass2',
       'addition_mass1', 'addition_mass2', 'ion_mass1', 'ion_mass2',
       'correct_mass1', 'correct_mass2', 'mass_difference1',
       'mass_difference2', 'm1+m2', '2m1+m2', 'm1+2m2', 'chosen_sum_from',
       'chosen_sum'],
      dtype='object')

In [196]:
df.head()

,ranking,each_original_data,ion1,loss1,loss_sign1,charge1,mass1,ion2,loss2,loss_sign2,...,ion_mass2,correct_mass1,correct_mass2,mass_difference1,mass_difference2,m1+m2,2m1+m2,m1+2m2,chosen_sum_from,chosen_sum
0,1,y7 (1+) @ 637.39 & b7 (1+) @ 690.36,y7,NaN,NaN,1+,637.39,b7,NaN,NaN,...,689.32451,637.305200,690.331760,0.084800,0.028240,1327.75,1965.14,2018.11,m1+m2,1327.75
1,2,y6 (1+) @ 580.39 & b8 (1+) @ 747.43,y6,NaN,NaN,1+,580.39,b8,NaN,NaN,...,746.34597,580.283740,747.353220,0.106260,0.076780,1327.82,1908.21,2075.25,m1+m2,1327.82
2,3,y5 (1+) @ 523.26 & b9 (1+) @ 804.37,y5,NaN,NaN,1+,523.26,b9,NaN,NaN,...,803.36743,523.262280,804.374680,-0.002280,-0.004680,1327.63,1850.89,2132.00,m1+m2,1327.63
3,4,y6 (1+) @ 580.37 & [b8-NH3] (1+) @ 730.35,y6,NaN,NaN,1+,580.37,b8,NH3,-,...,746.34597,580.283740,730.326671,0.086260,0.023329,1310.72,1891.09,2041.07,m1+m2,1310.72
4,5,[y4-NH3] (1+) @ 359.11 & b10 (1+) @ 951.13,y4,NH3,-,1+,359.11,b10,NaN,NaN,...,950.43584,359.167321,951.443090,-0.057321,-0.313090,1310.24,1669.35,2261.37,m1+m2,1310.24


In [94]:
def create_mass_conserve_line(row):
    ion1,ion2 = row['ion1'], row['ion2']
    loss1,loss2 = row['loss1'], row['loss2']
    if pd.isna(loss1):
        loss1 = ''
    if pd.isna(loss2):
        loss2 = ''
        
        
    #print((type(ion1) == str) and (ion1[0] == 'a' and ion1[:2] != 'ai'))
    #print((type(ion2) == str) and (ion2[0] == 'a' and ion2[:2] != 'ai')and (loss1 == '' and loss2 == ''))
    if (type(ion1) == str) and (ion1[0] == 'a' and ion1[:2] != 'ai') and (loss1 == '' and loss2 == ''):
        combined_loss = 'a'
    elif (type(ion2) == str) and (ion2[0] == 'a' and ion2[:2] != 'ai') and (loss1 == '' and loss2 == ''):
        combined_loss = 'a'
        
    elif loss1 != '' and loss2 != '':
        combined_loss = loss1 + ' + ' +loss2

    else:
        combined_loss = loss1 + loss2
    if combined_loss == '':
        combined_loss = 'Parent'
    return combined_loss


In [98]:
df['conserve_line'] = df.apply(create_mass_conserve_line, axis=1)

In [143]:
def peptie_arrange(length):
    result = []
    for i in range(1, length):
        result.append(f'b{i}y{length - i}')
    return result
        

In [179]:
rows = ['Parent','NH3','H2O', 'NH3 + H2O','H2O + NH3', 'a']
columns = peptie_arrange(14)
df['loss1_m'] = df['loss1'].fillna('')
df['loss2_m'] = df['loss2'].fillna('')

In [180]:
reuslt_df = pd.DataFrame(index=rows, columns=columns)

In [185]:
for index, each_row in df.iterrows():
    the_column = -1
    if (type(each_row['ion1']) == str and type(each_row['ion2']) == str):
        the_column = [each_row['ion1'], each_row['ion2']]
        the_column.sort()
        the_column = the_column[0] + the_column[1]
        the_column = the_column.replace('a', 'b')
    if (each_row['conserve_line'] in reuslt_df.index) and (the_column in reuslt_df.columns):
        print(each_row['conserve_line'])
        reuslt_df.at[each_row['conserve_line'], the_column] = each_row['ion1'] + each_row['loss1_m'] + each_row['ion2']+ each_row['loss2_m'] + ' \n ' +\
        each_row['chosen_sum_from'] + ' \n ' + str(round(each_row['mass_difference1'] + each_row['mass_difference2'], 2))
reuslt_df = reuslt_df.fillna('--')

Parent
Parent
Parent
NH3
NH3
NH3
NH3
NH3
a
NH3
H2O + NH3
Parent


In [186]:
reuslt_df

,b1y13,b2y12,b3y11,b4y10,b5y9,b6y8,b7y7,b8y6,b9y5,b10y4,b11y3,b12y2,b13y1
Parent,--,--,--,--,--,--,y7b7 \n m1+m2 \n 0.11,y6b8 \n m1+m2 \n 0.18,y5b9 \n m1+m2 \n -0.01,--,y3b11 \n m1+m2 \n -0.21,--,--
NH3,--,--,--,--,--,--,b7y7NH3 \n m1+m2 \n 0.87,y6NH3b8 \n m1+m2 \n 0.03,y5NH3b9 \n m1+m2 \n -0.02,y4NH3b10 \n m1+m2 \n -0.37,y3NH3b11 \n m1+m2 \n 0.31,--,--
H2O,--,--,--,--,--,--,--,--,--,--,--,--,--
NH3 + H2O,--,--,--,--,--,--,--,--,--,--,--,--,--
H2O + NH3,--,--,--,--,--,--,y7H2Ob7NH3 \n m1+m2 \n 0.18,--,--,--,--,--,--
a,--,--,--,--,--,--,--,y6a8 \n m1+m2 \n 0.4,--,--,--,--,--


In [163]:
df.columns

Index(['ranking', 'each_original_data', 'ion1', 'loss1', 'loss_sign1',
       'charge1', 'mass1', 'ion2', 'loss2', 'loss_sign2', 'charge2', 'mass2',
       'addition_mass1', 'addition_mass2', 'ion_mass1', 'ion_mass2',
       'correct_mass1', 'correct_mass2', 'mass_difference1',
       'mass_difference2', 'm1+m2', '2m1+m2', 'm1+2m2', 'chosen_sum_from',
       'chosen_sum', 'conserve_line'],
      dtype='object')

In [190]:
import pandas as pd
from html2image import Html2Image

# 1. Your original DataFrame and color dictionary
data = {
    'b12y1 | b2y1': ['(no loss,no loss) 0.31', '(no loss,NH3) -0.25', '(no loss,H2O)', '--'],
    'b3y10 | b4y9 | b5y8': ['--', '(NH3,no loss) -0.17', '--', '--'],
}
df = pd.DataFrame(data, index=['Parent', 'NH3', 'H2O', 'a'])

color_dict = {
    'NH3': 'blue',
    'H2O': 'red',
}

# 2. Create a copy of the DataFrame to modify its content
# It's good practice to work on a copy
df_display = df.copy().astype(str)

# 3. Iterate through the rows and inject HTML directly into the data
for index_name in df_display.index:
    if index_name in color_dict:
        color = color_dict[index_name]
        substring_to_color = index_name
        
        # Create the replacement HTML string
        replacement_html = f"<span style='color: {color}; font-weight: bold;'>{substring_to_color}</span>"
        
        # Modify the row in our display DataFrame
        # We use regex=False to ensure it treats the string literally
        df_display.loc[index_name] = df_display.loc[index_name].str.replace(
            substring_to_color, replacement_html, regex=False
        )

# 4. Now, apply simple, table-wide styles. No complex .apply() is needed.
styled_df = df_display.style.set_properties(**{
    'border': '1px solid black',
    'text-align': 'center'
})

# 5. Export to HTML (with escape=False) and then to PNG
html_output = styled_df.to_html(escape=False)
with open("colored_substrings.html", "w") as f:
    f.write(html_output)

hti = Html2Image()
hti.screenshot(html_file='colored_substrings.html', save_as='colored_substrings.png')

22134 bytes written to file /Users/kevinmbp/Desktop/2D_spec_dict/colored_substrings.png


['/Users/kevinmbp/Desktop/2D_spec_dict/colored_substrings.png']

In [192]:
import pandas as pd
from html2image import Html2Image

# Your setup code remains the same
data = {
    'b12y1 | b2y1': ['(no loss,no loss) 0.31', '(no loss,NH3) -0.25', '(no loss,H2O)', '--'],
    'b3y10 | b4y9 | b5y8': ['--', '(NH3,no loss) -0.17', '--', '--'],
}
df = pd.DataFrame(data, index=['Parent', 'NH3', 'H2O', 'a'])
color_dict = {
    'NH3': 'blue',
    'H2O': 'red',
    'Parent': 'green'
}
df_display = df.copy().astype(str)
for index_name in df_display.index:
    if index_name in color_dict:
        color = color_dict[index_name]
        replacement_html = f"<span style='color: {color}; font-weight: bold;'>{index_name}</span>"
        df_display.loc[index_name] = df_display.loc[index_name].str.replace(
            index_name, replacement_html, regex=False
        )

# 💡 THE FIX: Rewrite the function to handle a Series
def color_index_series(index_labels_series):
    """
    Takes a Series of index labels and returns a Series of CSS styles.
    """
    # Use .map() to apply the logic to each label in the Series
    return index_labels_series.map(
        lambda label: f"color: {color_dict.get(label, 'black')}; font-weight: bold;"
    )

# Now, call the new function
styled_df = df_display.style.apply_index(color_index_series, axis="index") \
                          .set_properties(**{
                              'border': '1px solid black',
                              'text-align': 'center'
                          })

# Exporting remains the same
html_output = styled_df.to_html(escape=False)
with open("colored_table_and_index.html", "w") as f:
    f.write(html_output)

hti = Html2Image()
hti.screenshot(html_file='colored_table_and_index.html', save_as='colored_table_and_index.png')

22220 bytes written to file /Users/kevinmbp/Desktop/2D_spec_dict/colored_table_and_index.png


['/Users/kevinmbp/Desktop/2D_spec_dict/colored_table_and_index.png']

In [197]:
import pandas as pd
from html2image import Html2Image

# 1. Setup with a new color dictionary to show the substring logic
data = {
    'Fragment 1': ['(no loss,no loss) 0.31', '(no loss,NH3) -0.25', '(no loss,H2O)', '(loss, H2O+NH3)'],
    'Fragment 2': ['--', '(NH3,no loss) -0.17', '--', '--'],
}
df = pd.DataFrame(data, index=['Parent', 'NH3', 'H2O', 'H2O+NH3'])

# 'NH3' will get its color from the 'H2O+NH3' key
color_dict = {
    'H2O': 'red',
    'H2O+NH3': 'purple'
}

# 2. Modify the DataFrame content based on substring logic
df_display = df.copy().astype(str)
for index_name in df_display.index:
    # Loop through the color dictionary to find a key that contains our index_name
    for dict_key, color in color_dict.items():
        if index_name in dict_key:
            # Found a match, create the replacement HTML
            replacement_html = f"<span style='color: {color}; font-weight: bold;'>{index_name}</span>"
            
            # Modify the row in our display DataFrame
            df_display.loc[index_name] = df_display.loc[index_name].str.replace(
                index_name, replacement_html, regex=False
            )
            # Break the inner loop to use the first match found
            break

# 3. Modify the index styling function for substring logic
def color_index_series(index_labels_series):
    
    def get_color_for_label(label):
        # Iterate through the dictionary to find the first matching key
        for dict_key, color in color_dict.items():
            if label in dict_key:
                # Return the style for the matched key's color
                return f"color: {color}; font-weight: bold;"
        # Default style if no key contains the label
        return "color: black;"

    # Map the helper function to each label in the index Series
    return index_labels_series.map(get_color_for_label)

# 4. Chain the styling methods and export
styled_df = df_display.style.apply_index(color_index_series, axis="index") \
                          .set_properties(**{
                              'border': '1px solid black',
                              'text-align': 'center'
                          })

html_output = styled_df.to_html(escape=False)
with open("colored_substring_logic.html", "w") as f:
    f.write(html_output)

hti = Html2Image()
hti.screenshot(html_file='colored_substring_logic.html', save_as='colored_substring_logic.png')

25000 bytes written to file /Users/kevinmbp/Desktop/2D_spec_dict/colored_substring_logic.png


['/Users/kevinmbp/Desktop/2D_spec_dict/colored_substring_logic.png']

In [200]:
import pandas as pd
from html2image import Html2Image

# 1. Setup with a DataFrame and a color dict that matches indices exactly
data = {
    'Fragment 1': ['(loss, Parent)', '(no loss,NH3)', '(no loss,H2O)', '(H2O,NH3)'],
    'Fragment 2': ['--', '(NH3,no loss)', '--', '--'],
}
df = pd.DataFrame(data, index=['Parent', 'NH3', 'H2O', 'H2O+NH3'])

# The keys in this dictionary must now be an EXACT match for an index label
color_dict = {
    'H2O': 'red',
    'NH3': 'blue',
    'H2O+NH3': 'purple'
}

# 2. Prepare a list of all possible substrings to find (which are the indices)
substrings_to_find = df.index.tolist()

# Create a copy of the DataFrame to modify its content
df_display = df.copy().astype(str)

# 3. Modify the DataFrame content using the new index-first logic
for row_index, row_series in df.iterrows():
    # Get the color for the CURRENT ROW's index
    row_color = color_dict.get(row_index)

    # If this row's index doesn't have a color defined, we do nothing and move on
    if not row_color:
        continue

    # For the current row, find ALL substrings that are also indices
    # and color them with the ROW's color.
    current_row_data = df_display.loc[row_index]
    for substring in substrings_to_find:
        replacement_html = f"<span style='color: {row_color}; font-weight: bold;'>{substring}</span>"
        current_row_data = current_row_data.str.replace(
            substring, replacement_html, regex=False
        )
    df_display.loc[row_index] = current_row_data


# 4. The index styling function remains the same, as it already uses exact matches
def color_index_series(index_labels_series):
    return index_labels_series.map(
        lambda label: f"color: {color_dict.get(label, 'black')}; font-weight: bold;"
    )

# 5. Chain the styling methods and export
styled_df = df_display.style.apply_index(color_index_series, axis="index") \
                          .set_properties(**{
                              'border': '1px solid black',
                              'text-align': 'center'
                          })

html_output = styled_df.to_html(escape=False)
with open("final_colored_table.html", "w") as f:
    f.write(html_output)

hti = Html2Image()
hti.screenshot(html_file='final_colored_table.html', save_as='final_colored_table.png')

21758 bytes written to file /Users/kevinmbp/Desktop/2D_spec_dict/final_colored_table.png


['/Users/kevinmbp/Desktop/2D_spec_dict/final_colored_table.png']

In [203]:
import pandas as pd
from html2image import Html2Image
import re # We'll use regular expressions to find components

# 1. Setup - I've recreated your data from the screenshot
data = {
    'b1y9': ['--', '--', '--', '--', '--', '--', '--'], 'b2y8': ['(b2,y8)\n(1+, 1+)\n-0.15 (32)', '--', '--', '--', '(H2O,NH3)\n(1+, 1+)\n-0.15 (32)', '--', '2'],
    'b3y7': ['(b3,y7)\n(1+, 1+)\n-0.02 (3)', '(b3,NH3)\n(1+, 1+)\n-0.62 (13)', '--', '--', '--', '--', '2'],
    'Row_Count': [8, 8, 0, 1, 0, 0, 17]
}
df = pd.DataFrame(data, index=['Parent', 'NH3', 'H2O', 'NH3 + H2O', 'H2O + NH3', 'a', 'Col_Count'])

# The color dictionary still uses exact index matches
color_dict = {
    'NH3': 'blue',
    'H2O': 'red',
    'NH3 + H2O': 'green',
    'H2O + NH3': 'green' # Handle both permutations
}

# 2. Define a helper function to find all components in a string
# This list defines what we consider a "component"
known_components = df.index.tolist()

def get_components(label_string):
    """Uses regex to find all known components within a string."""
    # Create a regex pattern like '(NH3|H2O|Parent|...)'
    pattern = '|'.join(re.escape(c) for c in known_components if c.strip())
    return re.findall(pattern, label_string)

# 3. Modify the DataFrame using the new component-based logic
df_display = df.copy().astype(str).map(lambda x: x.replace('\n', '<br>')) # Handle newlines first

for row_index, row_series in df.iterrows():
    row_color = color_dict.get(row_index)
    if not row_color:
        continue
        
    # Get the fundamental components for THIS row
    components_to_color = get_components(row_index)
    
    # 💡 Sort by length (descending) to replace 'H2O+NH3' before 'H2O'
    components_to_color.sort(key=len, reverse=True)
    
    current_row_data = df_display.loc[row_index]
    for component in components_to_color:
        replacement_html = f"<span style='color: {row_color}; font-weight: bold;'>{component}</span>"
        current_row_data = current_row_data.str.replace(
            component, replacement_html, regex=False
        )
    df_display.loc[row_index] = current_row_data

# 4. The index styling function remains the same
def color_index_series(index_labels_series):
    return index_labels_series.map(
        lambda label: f"color: {color_dict.get(label, 'black')}; font-weight: bold;"
    )

# 5. Chain the styling methods and export
styled_df = df_display.style.apply_index(color_index_series, axis="index") \
                          .set_properties(**{
                              'border': '1px solid black',
                              'text-align': 'center'
                          })

html_output = styled_df.to_html(escape=False)
with open("final_complex_table.html", "w") as f:
    f.write(html_output)

from html2image import Html2Image
hti = Html2Image()
hti.screenshot(html_file='final_complex_table.html', save_as='final_complex_table.png')

31453 bytes written to file /Users/kevinmbp/Desktop/2D_spec_dict/final_complex_table.png


['/Users/kevinmbp/Desktop/2D_spec_dict/final_complex_table.png']